In [15]:
# import libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.autograd import Variable
import random
import math


#####################
# Load Data
#####################

# load data set_v1
excel_file = 'image-manipulation (timeseries)/Caldwell_ImageManipulation-EyeGaze_DataSetCombined.xlsx'
olddata = pd.read_excel(excel_file, sheet_name=1)
# rename the labels of dataset_v1
olddata.rename(columns={'participant':'Participant_ID'}, inplace=True)
olddata.rename(columns={'image':'Image_ID'}, inplace=True)
# load data set_2
excel_file = 'image-manipulation_v2/Caldwell_Manip_Images_10-14_TimeSeries.csv'
predata = pd.read_csv(excel_file)
# merge two dataset by the 'Participant_ID','Image_ID'
predata2 = pd.merge(predata,olddata,how='left', on=['Participant_ID','Image_ID'])
# remove the useless column from dataset_v1
newdata = predata2.iloc[:, [0,1,2,3,4,5,6,7,8,13,14]]
# sort the dataset by start time
data = newdata.sort_values(by=['Start Time'])

# normalization the dataset between 0 and 1 by Min-Max scaling
# https://sebastianraschka.com/Articles/2014_about_feature_scaling.html#about-min-max-scaling
for i in range(0, data.shape[1] - 2):
    dataMax = data[[data.columns[i]]].max()
    dataMin = data[[data.columns[i]]].min()
    data[[data.columns[i]]] = (data[[data.columns[i]]] - dataMin)/ (dataMax - dataMin)

# use the Image_ID 10-12 (after normalization 0-0.5)as the train data,13 (0.75) as validate data and 14 (1.0) as test data
traindata = data.loc[data['Image_ID'] < 0.75]

validatedata = data.loc[data['Image_ID'] == 0.75]

testdata = data.loc[data['Image_ID'] == 1.0]

train_input = traindata.iloc[:, :9]
train_target = traindata.iloc[:, -1]

validate_input = validatedata.iloc[:, :9]
validate_target = validatedata.iloc[:, -1]

test_input = testdata.iloc[:, :9]
test_target = testdata.iloc[:, -1]

X_train = Variable(torch.Tensor(train_input.values).float())
Y_train = Variable(torch.Tensor(train_target.values).long())

X_validate = Variable(torch.Tensor(validate_input.values).float())
Y_validate = Variable(torch.Tensor(validate_target.values).long())

X_test = Variable(torch.Tensor(test_input.values).float())
Y_test = Variable(torch.Tensor(test_target.values).long())

In [16]:
#####################
# LSTM model
#####################

n_features = 9
num_epochs = 1000
learning_rate = 0.1
batch_size = 1
#num_hidden_layer = 1
input_dim = n_features
hidden_dim = 10
output_dim=3
num_layers=1

# reference example LSTM
# https://www.jessicayung.com/lstms-for-time-series-in-pytorch/
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, batch_size, output_dim, num_layers, batch_first=True):
        super(LSTMModel, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.num_layers = num_layers
        
        # Define the LSTM layer
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, self.num_layers, batch_first=True)
        
        # Define the output layer
        self.fc = nn.Linear(self.hidden_dim, output_dim)
        
        
    def init_hidden(self):
        # This is what we'll initialise our hidden state as
        return (torch.zeros(self.num_layers, self.batch_size, self.hidden_dim),
                torch.zeros(self.num_layers, self.batch_size, self.hidden_dim))
    
    def forward(self, input):
        #input = input.unsqueeze(0)

        # Initialize hidden state with zeros
        #h0 = torch.zeros(self.num_layers, input.size(0), self.hidden_dim).requires_grad_()

        # Initialize cell state
        #c0 = torch.zeros(self.num_layers, input.size(0), self.hidden_dim).requires_grad_()

        # Forward pass through LSTM layer
        # shape of lstm_out: [input_size, batch_size, hidden_dim]
        # shape of self.hidden: (a, b), where a and b both 
        # have shape (num_layers, batch_size, hidden_dim).
        
        out, self.hidden = self.lstm(input.view(len(input), self.batch_size, -1))
        #out, self.hidden = self.lstm(input)
        #out, (hn, cn) = self.lstm(input, (h0.detach(), c0.detach()))

        
        # Only take the output from the final timetep
        # Can pass on the entirety of lstm_out to the next layer if it is a seq2seq prediction
        #out = self.fc(out[-1].view(self.batch_size, -1))
        out = self.fc(out[:, -1, :]) 
        out = F.softmax(out, dim = 1)
        return out


        

model = LSTMModel(input_dim, hidden_dim, batch_size, output_dim, num_layers)

model

LSTMModel(
  (lstm): LSTM(9, 10, batch_first=True)
  (fc): Linear(in_features=10, out_features=3, bias=True)
)

In [17]:
# store all losses for visualisation
all_losses = []
optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate)
v_losses = []
"""
Train the neural network

"""

# train a neural network
for epoch in range(num_epochs):
    # Initialise hidden state
    model.hidden = model.init_hidden()

    # Perform forward pass: compute predicted y by passing x to the model.
    Y_pred = model(X_train)
    
    Y_pred_validate = model(X_validate)
    
    loss_func = torch.nn.CrossEntropyLoss()
    #loss_func = torch.nn.MSELoss(size_average=False)


    # Compute loss
    loss = loss_func(Y_pred, Y_train)
    all_losses.append(loss.item())
    
        
    lossv = loss_func(Y_pred_validate,Y_validate)
    v_losses.append(lossv.item())


    # Clear the gradients before running the backward pass.
    model.zero_grad()

    # Perform backward pass
    loss.backward()

    # Calling the step function on an Optimiser makes an update to its
    # parameters
    optimiser.step()
    
    
    if epoch % 100 == 0:
        # convert three-column predicted Y values to one column for comparison
        _, predicted = torch.max(Y_pred, 1)

        # calculate and print accuracy
        total = predicted.size(0)
        correct = predicted.data.numpy() == Y_train.data.numpy()

        #print('Epoch [%d/%d] Loss: %.4f  Accuracy: %.2f %%'
        #      % (epoch + 1, num_epochs, loss.item(), 100 * sum(correct)/total))
        
        print('Epoch [%d/%d] Loss: %.4f  Accuracy: %.2f %%'
              % (epoch + 1, num_epochs, loss.item(), 100 * sum(correct)/total))
        
        
        # validate set
        _, predictedv = torch.max(Y_pred_validate, 1)
                
        totalv = predictedv.size(0)
        correctv = predictedv.data.numpy() == Y_validate.data.numpy()
        
        print('Validation Epoch [%d/%d] Loss: %.4f  Accuracy: %.2f %%'
              % (epoch + 1, num_epochs, lossv.item(), 100 * sum(correctv)/totalv))
        
"""
Test the neural network

Pass testing data to the built neural network and get its performance
"""
# test the neural network using testing data
# It is actually performing a forward pass computation of predicted y
# by passing x to the model.
# Here, Y_pred_test contains three columns, where the index of the
# max column indicates the class of the instance
Y_pred_test = model(X_test)
# get prediction
# convert three-column predicted Y values to one column for comparison
_, predicted_test = torch.max(Y_pred_test, 1)

# calculate accuracy
total_test = predicted_test.size(0)
correct_test = sum(predicted_test.data.numpy() == Y_test.data.numpy())
print('Testing Accuracy: %.2f %%' % (100 * correct_test / total_test))

#print loss fig

plt.xlabel("Iteration")
plt.ylabel("Loss (CrossEntropyLoss)")

plt.plot(all_losses,label="Train_losses")
plt.plot(v_losses, label="Validate_losses")
plt.legend()
plt.figure()
plt.show()



Epoch [1/1000] Loss: 1.0935  Accuracy: 45.24 %
Validation Epoch [1/1000] Loss: 1.1142  Accuracy: 36.55 %
Epoch [101/1000] Loss: 0.8274  Accuracy: 72.94 %
Validation Epoch [101/1000] Loss: 1.1508  Accuracy: 39.39 %
Epoch [201/1000] Loss: 0.8038  Accuracy: 75.39 %
Validation Epoch [201/1000] Loss: 1.1679  Accuracy: 37.73 %


KeyboardInterrupt: 